In [369]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [320]:
def timeToSlot(times, initTime = pd.to_datetime('2016-01-01 00:00:00')):
    res = (times-initTime).astype('timedelta64[s]')
    return(res/600)
    
a = timeToSlot(weather.TIME)
a = a.round().astype('int64')
a = a.drop_duplicates()
len(a)
max(a)
min(a)

0

In [466]:
# fill in 0 where is missing value for GAP in the Orders
train = pd.read_csv("trainGAP.csv")
train['DATE'] = pd.to_datetime(train['DATE'])

DISTRICT_HASH = pd.DataFrame( {'DISTRICT_HASH': train['DISTRICT_HASH'].drop_duplicates(),
                              'key':1})
DATE = pd.DataFrame( {'DATE' : pd.date_range('20160101',periods = 21), 'key' : 1})
SLOTID = pd.DataFrame( { 'SLOTID': range(144),'key':1})
merged = pd.merge(DISTRICT_ID, DATE, on='key')
merged = pd.merge(merged, SLOTID, on='key')
len(merged['DATE'])
merged = merged[['DISTRICT_HASH','DATE','SLOTID']]
merged2 = pd.merge(merged,train, how='left', on = ['DISTRICT_HASH','DATE','SLOTID'])
merged2['GAP']= merged2['GAP'].fillna(0)
train = merged2
train = train [['DISTRICT_HASH', 'DATE', 'SLOTID', 'GAP']]
del merged, merged2



In [446]:
#fill in the nearest-time value 
Weather = pd.read_csv('weather.csv')
Weather["WEATHER"] = Weather["WEATHER"].astype("category")
Weather['DATE'] = pd.to_datetime(Weather['DATE'])
Weahter = Weather.sort(columns = ['DATE','SLOTID'])
DATE = pd.DataFrame( {'DATE' : pd.date_range('20160101',periods = 21), 'key' : 1})
SLOTID = pd.DataFrame( { 'SLOTID': range(144),'key':1})
merged = pd.merge(DATE,SLOTID, on='key')
merged = merged[['DATE','SLOTID']]
merged2 = pd.merge(merged,Weather, how = 'left',on= ['DATE','SLOTID'])

isMissing = pd.isnull(merged2['WEATHER'])
first = 0
if isMissing[first]:
    first = first + 1 
for i in range(first):
    merged2.set_value(1,'WEATHER',merged2.iloc[first,]['WEATHER'])
    merged2.set_value(1,'TEMPERATURE',merged2.iloc[first,]['TEMPERATURE'])
    merged2.set_value(1,'PM25',merged2.iloc[first,]['PM25'])
for i in range(len(merged2['DATE'])):
    if isMissing[i]:
        merged2.set_value(i,'WEATHER',row['WEATHER'])
        merged2.set_value(i,'TEMPERATURE',row['TEMPERATURE'])
        merged2.set_value(i,'PM25', row['PM25'])
Weather = merged2
del merged, merged2

isMissing.sum()

354

In [445]:
isMissing = pd.isnull(weather['WEATHER'])
isMissing.sum()

0

In [447]:
POI2 = pd.read_csv('POI2.csv')
DistMapping = pd.read_csv('DistMapping.csv')
Traffic = pd.read_csv("Traffic.csv")
Traffic['DATE'] = pd.to_datetime(Traffic['DATE'])
Traffic.columns

Index([u'DISTRICT_HASH', u'TJ_LEVEL1', u'TJ_LEVEL2', u'TJ_LEVEL3',
       u'TJ_LEVEL4', u'TJ_TIME', u'DATE', u'SLOTID'],
      dtype='object')

In [448]:
# fill the missing of the traffic
DISTRICT_ID = pd.DataFrame( {'DISTRICT_HASH': train['DISTRICT_HASH'].drop_duplicates(),
                              'key':1})
DATE = pd.DataFrame( {'DATE' : pd.date_range('20160101',periods = 21), 'key' : 1})
SLOTID = pd.DataFrame( { 'SLOTID': range(144),'key':1})
merged = pd.merge(DISTRICT_HASH, DATE, on='key')
merged = pd.merge(merged, SLOTID, on='key')
merged = merged[['DISTRICT_HASH','DATE','SLOTID']]
merged2= pd.merge(merged, Traffic, how ='left', on =['DISTRICT_HASH','DATE','SLOTID'] )
len(merged2['DATE'])

isMissing = pd.isnull(merged2['TJ_LEVEL1'])
first = 0
if isMissing[first]:
    first = first + 1 
merged2.set_value(1,'TJ_LEVEL1',merged2.iloc[first,]['TJ_LEVEL1'])
merged2.set_value(1,'TJ_LEVEL2',merged2.iloc[first,]['TJ_LEVEL2'])
merged2.set_value(1,'TJ_LEVEL3',merged2.iloc[first,]['TJ_LEVEL3'])
merged2.set_value(1,'TJ_LEVEL4',merged2.iloc[first,]['TJ_LEVEL4'])  
merged2.set_value(1,'TJ_TIME',merged2.iloc[first,]['TJ_TIME']) 
for i in range(len(merged2['DATE'])):
    if isMissing[i]:
        merged2.set_value(i,'TJ_LEVEL1',merged2.iloc[i-1,]['TJ_LEVEL1'])
        merged2.set_value(i,'TJ_LEVEL2',merged2.iloc[i-1,]['TJ_LEVEL2'])
        merged2.set_value(i,'TJ_LEVEL3',merged2.iloc[i-1,]['TJ_LEVEL3'])
        merged2.set_value(i,'TJ_LEVEL4',merged2.iloc[i-1,]['TJ_LEVEL4'])  
        merged2.set_value(i,'TJ_TIME',merged2.iloc[i-1,]['TJ_TIME'])
Traffic = merged2
del merged, merged2
isMissing.sum()


6031

In [458]:
isMissing = pd.isnull(Traffic['TJ_LEVEL1'])
isMissing.sum()


0

In [467]:
train1 = pd.merge(train, Weather, how = 'left', on = ['DATE','SLOTID'])
train1 = pd.merge(train1,POI2, how = 'left', on = 'DISTRICT_HASH')
train1 = pd.merge(DistMapping, train1, on = 'DISTRICT_HASH')
train1 = pd.merge( train1, Traffic, how = 'left' ,on = ['DISTRICT_HASH','DATE','SLOTID'])


pd.isnull(train1).sum()

DISTRICT_HASH    0
DATE             0
SLOTID           0
GAP              0
dtype: int64

In [470]:
feature_names = train1.columns[:]
variables_removed = ['DISTRICT_HASH','ID','TJ_TIME']
feature_names = feature_names[~ feature_names.isin(variables_removed)] # remove some IDs
feature_names

Index([u'DATE', u'SLOTID', u'GAP', u'WEATHER', u'TEMPERATURE', u'PM25',
       u'POI17X4', u'POI4X15', u'POI17X3', u'POI17X2', 
       ...
       u'POI20X9', u'POI20X8', u'POI6X3', u'POI8X2', u'POI8X3', u'POI1X1',
       u'TJ_LEVEL1', u'TJ_LEVEL2', u'TJ_LEVEL3', u'TJ_LEVEL4'],
      dtype='object', length=186)

Index([u'DISTRICT_HASH', u'ID'], dtype='object')

In [27]:
test = 

Index([u'GAP', u'ID', u'DATE', u'SLOTID', u'WEATHER', u'PM25', u'TEMPERATURE',
       u'TJ_LEVEL1', u'TJ_LEVEL2', u'TJ_LEVEL3', 
       ...
       u'POI20X3', u'POI20X2', u'POI20X9', u'POI20X8', u'POI6X3', u'POI8X2',
       u'POI8X3', u'POI1X1', u'ISWEEKEND', u'ISHOLIDAY'],
      dtype='object', length=189)

In [33]:
test=test.rename(columns = {'ID':'DISTRICT_ID'})

In [34]:
test.columns

Index([u'GAP', u'DISTRICT_ID', u'DATE', u'SLOTID', u'WEATHER', u'PM25',
       u'TEMPERATURE', u'TJ_LEVEL1', u'TJ_LEVEL2', u'TJ_LEVEL3', 
       ...
       u'POI20X3', u'POI20X2', u'POI20X9', u'POI20X8', u'POI6X3', u'POI8X2',
       u'POI8X3', u'POI1X1', u'ISWEEKEND', u'ISHOLIDAY'],
      dtype='object', length=189)

In [39]:
test = test.sort(columns= ['DISTRICT_ID', 'DATE', 'SLOTID'])
train = train.sort(columns= ['DISTRICT_ID', 'DATE', 'SLOTID'])


In [42]:
# create the first guess prediction: with the previous gap

timeID = []
distID = []
pred = []
#test1 = test[['ID','DATE','SLOTID','GAP']]
for i in range(2,len(test['DISTRICT_ID']),3):
    row = test.iloc[i,:]
    distID.append(row['DISTRICT_ID'])
    timeID.append(str(row['DATE']) + '-' +str(row['SLOTID']+2)) # 
    pred.append(row['GAP'])
submitResult =pd.DataFrame({'distID': distID,
                            'timeID': timeID,
                            'GAP': pred}) 
submitResult = submitResult[['distID','timeID','GAP']]



In [50]:
GAP_tr = train['GAP']
GAP_te = test['GAP']
feature_names = train.columns[1:]
IDs = ['DISTRICT_ID','DATE']
feature_names = feature_names[~ feature_names.isin(IDs)] # remove some IDs
feature_names[:]
features_tr = train[feature_names]
features_te = test[feature_names]
features_tr['SLOTID'][144:288]

#timeslot is continous
len(train)

102680

In [54]:
len(train['DISTRICT_ID'].drop_duplicates()) #66
len(train['DATE'].drop_duplicates()) # 21
len(train['SLOTID'].drop_duplicates()) #144

144

In [55]:
66*21*144

199584

In [ ]:
#need the continuous observations:

In [58]:
#fill in 0 when there is a missing observations:
submit = pd.read_csv('./test_set_1/read_me_1.txt')

In [7]:
submitResult.dtypes
submitResult.iloc[0:10,:]

,distID,timeID,GAP
0,1,2016-01-22-46,5
1,1,2016-01-22-58,11
2,1,2016-01-22-70,1
3,1,2016-01-22-82,5
4,1,2016-01-22-94,3
5,1,2016-01-22-106,3
6,1,2016-01-22-118,7
7,1,2016-01-22-130,8
8,1,2016-01-22-142,12
9,1,2016-01-24-68,9


In [133]:
submitResult.to_csv("result1.csv")

In [73]:
from math import fabs 
def Accuracy (GAP, pred):
    s = 0;
    for i in range(len(GAP)):
        if GAP[i] > 0:
            s = s + fabs( (GAP[i]-pred[i])*1.0/GAP[i])
    return s/len(GAP) 


        

In [80]:
from sklearn import tree

clf = tree.DecisionTreeRegressor()
clf = clf.fit(features_tr, GAP_tr)
scores = cross_val_score(clf, X, y)
ypred_tree = clf.predict(features_te)
Accuracy(GAP_te,ypred_tree)
scores

NameError: name 'cross_val_score' is not defined

In [79]:
from sklearn.cross_validation import cross_val_score
from sklearn.tree import DecisionTreeRegressor
clf = DecisionTreeRegressor(max_depth=None, min_samples_split=5, 
                             random_state=0)
clf = clf.fit(features_tr, GAP_tr)
ypred_dt = clf.predict(features_te)
ypred[:10]
Accuracy(GAP_te,ypred_dt)

1.507444550992227

In [77]:
from sklearn.ensemble import RandomForestClassifier


0.8709405503389853